In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
import os

# Defining paths

In [3]:
train_path = "/Users/andrei/code/images/images/train"

In [4]:
test_path = "/Users/andrei/code/images/images/validation"

In [5]:
val_path = "~/code/images/images/val"

# Loading the images

In [6]:
# iterate through the names of contents of the folder
all_images = []
y_train = []
X_train = []
for folder_path in os.listdir(train_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(train_path, folder_path)):
            img = load_img(os.path.join(os.path.join(train_path, folder_path), image_path), color_mode = "grayscale")
            X_train.append(img_to_array(img))
            y_train.append(os.path.basename(os.path.normpath(folder_path)))

In [7]:
y_train = np.array(y_train)

In [8]:
X_train = np.array(X_train)

## Test


In [9]:
# iterate through the names of contents of the folder
all_images = []
y_test = []
X_test = []

for folder_path in os.listdir(test_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(test_path, folder_path)):
            img = load_img(os.path.join(os.path.join(test_path, folder_path), image_path), color_mode = "grayscale")
            X_test.append(img_to_array(img))
            y_test.append(os.path.basename(os.path.normpath(folder_path)))

In [10]:
y_test = np.array(y_test)

In [11]:
X_test = np.array(X_test)

# Investigation and baseline

In [12]:
y_train_df = pd.DataFrame(y_train)

In [13]:
y_test_df = pd.DataFrame(y_test)

In [14]:
y_train_df.value_counts(normalize=True)

happy       0.248569
neutral     0.172860
sad         0.171333
fear        0.142361
angry       0.138545
surprise    0.111204
disgust     0.015128
dtype: float64

In [15]:
y_test_df.value_counts(normalize=True)

happy       0.258279
neutral     0.172092
sad         0.161194
fear        0.144070
angry       0.135862
surprise    0.112794
disgust     0.015709
dtype: float64

In [16]:
baseline = y_train_df.value_counts(normalize=True).max()

In [17]:
baseline

0.2485687519517019

In [18]:
X_train.shape

(28821, 48, 48, 1)

In [19]:
X_test.shape

(7066, 48, 48, 1)

# Preprocessing

In [20]:
from sklearn.preprocessing import OneHotEncoder

In [21]:
ohe = OneHotEncoder(sparse = False)

In [22]:
y_train_reshaped = y_train.reshape(-1, 1)

In [23]:
y_test_reshaped = y_test.reshape(-1, 1)

In [24]:
y_train_cat = ohe.fit_transform(y_train_reshaped)

In [25]:
y_test_cat = ohe.transform(y_test_reshaped)

In [26]:
ohe.get_feature_names_out()

array(['x0_angry', 'x0_disgust', 'x0_fear', 'x0_happy', 'x0_neutral',
       'x0_sad', 'x0_surprise'], dtype=object)

In [27]:
y_train_cat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [28]:
X_train.shape

(28821, 48, 48, 1)

In [29]:
from sklearn.utils import shuffle

In [30]:
X_shuffle, y_shuffle = shuffle(X_train, y_train_cat, random_state = 0)

In [31]:
X_sample = X_shuffle[:3000, :, :, :]

In [32]:
y_sample = y_shuffle[:3000]

# Model

In [37]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers, models

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers, models

model = models.Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model.add(Rescaling(1./255, input_shape=(48, 48, 1)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model.add(layers.Conv2D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=2, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [34]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model.fit(X_sample, y_sample, batch_size = 32, epochs = 1000,
          callbacks=[EarlyStopping(patience = 20, restore_best_weights= True, monitor = "val_accuracy", mode = "max")],
         validation_split = 0.2, verbose = 1)

In [ ]:
model.evaluate(X_test, y_test_cat)

# Fitting on all of the data

In [38]:
X_shuffle_all, y_shuffle_all = shuffle(X_train, y_train_cat, random_state = 0)

In [39]:
model2 = models.Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model2.add(Rescaling(1./255, input_shape=(48, 48, 1)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model2.add(layers.Conv2D(16, kernel_size=5, activation='relu'))
model2.add(layers.MaxPooling2D(3))

model2.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
model2.add(layers.MaxPooling2D(3))

model2.add(layers.Conv2D(32, kernel_size=2, activation="relu"))
model2.add(layers.MaxPooling2D(3))

model2.add(layers.Flatten())
model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(7, activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2022-09-06 11:40:22.833196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [40]:
model2.fit(X_shuffle_all, y_shuffle_all, batch_size = 32, epochs = 1000,
          callbacks=[EarlyStopping(patience = 50, restore_best_weights= True, monitor = "val_accuracy", mode = "max")],
         validation_split = 0.2, verbose = 1)

Epoch 1/1000
721/721 [==============================] - 10s 13ms/step - loss: 1.8065 - accuracy: 0.2503 - val_loss: 1.7812 - val_accuracy: 0.2602
Epoch 2/1000
721/721 [==============================] - 9s 12ms/step - loss: 1.7330 - accuracy: 0.2860 - val_loss: 1.6955 - val_accuracy: 0.3211
Epoch 3/1000
721/721 [==============================] - 9s 13ms/step - loss: 1.6286 - accuracy: 0.3508 - val_loss: 1.5999 - val_accuracy: 0.3690
Epoch 4/1000
721/721 [==============================] - 9s 12ms/step - loss: 1.5370 - accuracy: 0.4024 - val_loss: 1.5608 - val_accuracy: 0.3967
Epoch 5/1000
721/721 [==============================] - 10s 14ms/step - loss: 1.4799 - accuracy: 0.4288 - val_loss: 1.4901 - val_accuracy: 0.4300
Epoch 6/1000
721/721 [==============================] - 11s 15ms/step - loss: 1.4293 - accuracy: 0.4496 - val_loss: 1.4607 - val_accuracy: 0.4397
Epoch 7/1000
721/721 [==============================] - 10s 14ms/step - loss: 1.3918 - accuracy: 0.4619 - val_loss: 1.4285 - va

Epoch 57/1000
721/721 [==============================] - 9s 13ms/step - loss: 0.9440 - accuracy: 0.6429 - val_loss: 1.4811 - val_accuracy: 0.4876
Epoch 58/1000
721/721 [==============================] - 10s 13ms/step - loss: 0.9455 - accuracy: 0.6413 - val_loss: 1.4819 - val_accuracy: 0.4723
Epoch 59/1000
721/721 [==============================] - 10s 14ms/step - loss: 0.9410 - accuracy: 0.6415 - val_loss: 1.5000 - val_accuracy: 0.4727
Epoch 60/1000
721/721 [==============================] - 10s 13ms/step - loss: 0.9337 - accuracy: 0.6443 - val_loss: 1.4697 - val_accuracy: 0.4885
Epoch 61/1000
721/721 [==============================] - 9s 13ms/step - loss: 0.9326 - accuracy: 0.6453 - val_loss: 1.5057 - val_accuracy: 0.4853
Epoch 62/1000
721/721 [==============================] - 10s 13ms/step - loss: 0.9341 - accuracy: 0.6459 - val_loss: 1.4710 - val_accuracy: 0.4900
Epoch 63/1000
721/721 [==============================] - 10s 14ms/step - loss: 0.9277 - accuracy: 0.6471 - val_loss: 1.4

In [ ]:
model2.evaluate(X_test, y_test_cat)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
output = ConfusionMatrixDisplay(confusion_matrix(np.argmax(y_test_cat, axis =1), np.argmax(y_pred, axis =1)))

In [ ]:
output.plot()

In [ ]:
y_test

In [ ]:
ohe.get_feature_names_out()

In [ ]:
y_test

# Uploading a temp model to mlflow

In [41]:
import mlflow

In [46]:
import time

In [49]:
def save_model(model: Model = None,
               params: dict = None,
               metrics: dict = None) -> None:
    """
    persist trained model, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    # retrieve mlflow env params
    mlflow_tracking_uri = "https://mlflow.lewagon.ai"
    mlflow_experiment = "[batch-960]-job_prepr_temp"
    mlflow_model_name = "[batch-960]-job_prepr_temp"

    # configure mlflow
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name=mlflow_experiment)

    with mlflow.start_run():

        mlflow.keras.log_model(keras_model=model,
                                   artifact_path="model",
                                   keras_module="tensorflow.keras",
                                   registered_model_name=mlflow_model_name)

    print("\n✅ data saved to mlflow")

    return None

In [50]:
save_model(model2)

2022/09/06 11:55:39 INFO mlflow.tracking.fluent: Experiment with name '[batch-960]-job_prepr_temp' does not exist. Creating a new experiment.


INFO:tensorflow:Assets written to: /var/folders/lg/x7r61bz54jlf6d0nvwrlqd0m0000gn/T/tmp8whg0dcx/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/lg/x7r61bz54jlf6d0nvwrlqd0m0000gn/T/tmp8whg0dcx/model/data/model/assets
Successfully registered model '[batch-960]-job_prepr_temp'.
2022/09/06 11:55:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: [batch-960]-job_prepr_temp, version 1



✅ data saved to mlflow


Created version '1' of model '[batch-960]-job_prepr_temp'.


In [3]:
import mlflow

In [4]:
mlflow.set_tracking_uri('https://mlflow.lewagon.ai/')
mlflow_model_name = '[batch-960]-job_prepr_temp'
model_uri = f"models:/{mlflow_model_name}/Production"
try:
    model = mlflow.keras.load_model(model_uri=model_uri)
except:
    raise Exception(f'No model in Production on mlflow')

2022-09-06 12:56:31.463919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 48, 48, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 44, 44, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 32)          4

In [8]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 48, 48, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'rescaling_input'}},
  {'class_name': 'Rescaling',
   'config': {'name': 'rescaling',
    'trainable': True,
    'batch_input_shape': (None, 48, 48, 1),
    'dtype': 'float32',
    'scale': 0.00392156862745098,
    'offset': 0.0}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d',
    'trainable': True,
    'dtype': 'float32',
    'filters': 16,
    'kernel_size': (5, 5),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': 